#Loading recent negative reviews from STEAM

In [ ]:
!pip install steamreviews

In [38]:
import steamreviews # https://pypi.org/project/steamreviews/
import pandas as pd


In [94]:
app_id = 420 #Half-life 2

# Reference: https://partner.steamgames.com/doc/store/getreviews
request_params = dict()
request_params['language'] = 'english'
request_params['review_type'] = 'negative'
request_params['purchase_type'] = 'steam'
request_params['filter'] = 'recent'
request_params['day_range'] = '28'

review_dict, query_count = steamreviews.download_reviews_for_app_id(app_id, chosen_request_params=request_params)

[appID = 420] expected #reviews = 380


In [95]:
review_dict['reviews'][list(review_dict['reviews'].keys())[0]]

{'recommendationid': '133699736',
 'author': {'steamid': '76561198864989974',
  'num_games_owned': 50,
  'num_reviews': 21,
  'playtime_forever': 365,
  'playtime_last_two_weeks': 0,
  'playtime_at_review': 365,
  'last_played': 1677448845},
 'language': 'english',
 'review': 'I gave up on the part where you need to defend against striders...a horrible level design, it was really frustrating, so I never finished this DLC\notherwise its was an excellent DLC and I would give it 9.5/10, muuuch better than the first one',
 'timestamp_created': 1677449499,
 'timestamp_updated': 1677449499,
 'voted_up': False,
 'votes_up': 0,
 'votes_funny': 1,
 'weighted_vote_score': '0.446927368640899658',
 'comment_count': 0,
 'steam_purchase': True,
 'received_for_free': False,
 'written_during_early_access': False,
 'hidden_in_steam_china': True,
 'steam_china_location': ''}

In [96]:
len(review_dict['reviews'])

380

In [124]:
# remove the fields we wont use
df = pd.DataFrame(review_dict['reviews'].values(), )
df = df[['recommendationid', 'review' ]]
df.head()

,recommendationid,review
0,133699736,I gave up on the part where you need to defend...
1,133400321,Lamarr died
2,133383836,egg
3,132852329,I'm glad that I waited for so long to finally ...
4,132698796,kinda sucks since they refuse to continue the ...


# Sending them to GPT

In [ ]:
!pip install --upgrade pip
!pip install setuptools wheel
!pip install openai

In [63]:
import openai 

In [146]:
#os.environ["OPENAI_API_KEY"] = 'API KEY HERE'

In [65]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [147]:
def make_review_prompt(reviews_list, max_number_of_reviews = 50):
  '''
    Returns a GPT prompt asking to analyse a list of steam reviews and provide the 10 most common issues.

            Parameters:
                    reviews_list : a list of reviews generated with steamreviews
                    max_number_of_reviews : the maximm number of reviews that will be included in the prompt

            Returns:
                    prompt (str): a prompt for GPT
    '''
  reviews = ''

  if max_number_of_reviews > len(reviews_list):
     max_number_of_reviews = len(reviews_list);

  for i in range(max_number_of_reviews):
    reviews += "\n\nreview_id : " + reviews_list.recommendationid[i]  + "\nreview_text : " + reviews_list.review[i]
  
  return '''
    Read these reviews and tell me what are the 10 most common issues reported. 
    For each issue, provide relevant review_id(s) and keywords associated with the issue. 
    Reviews:''' + reviews + "\n\nThe 10 most common issues reported: "

In [148]:
prompt = make_review_prompt(df, 35)

In [144]:
response = openai.Completion.create(
    model='text-davinci-003',
    prompt=prompt, 
    max_tokens=1000)

print(response['choices'][0]['text'])



1. Unfinished product (review_id: 129080634, 128777479, 128632453, 127908755) - Keywords: unfinished, cliffhanger, incomplete, unresolved
2. Poor level design (review_id: 133699736) - Keywords: horrible, frustrating
3. Obnoxious characters/NPCs (review_id: 132698796, 130110119, 128767749) - Keywords: obnoxious, nagging
4. Confusing FoV settings (review_id: 133383836) - Keywords: FoV
5. Overuse of the trope "Oh, everyone always knew all about this but just never mentioned it" (review_id: 132852329) - Keywords: always knew, never mentioned
6. Lackluster/repetitive gameplay (review_id: 130846992, 127090766, 127908755, 123225118, 120913424, 120254438, 119893438) - Keywords: repetitive, lackluster, boring
7. Poor integration of Portal (review_id: 132852329) - Keywords: Portal, integration
8. Strider battle is hard/frustrating (review_id: 133699736, 132176313, 119419451) - Keywords: strider, hard, frustrating
9. Unnecessary grind (review_id: 128277724) - Keywords: grind
10. Difficult/unkno